# Telco Customer Churn

**by Linh Toan**<br/>
**Data Science @ Newman University**

**Data:** Telco customer churn: IBM dataset from kaggle - https://www.kaggle.com/datasets/yeanzc/telco-customer-churn-ibm-dataset?resource=download.

**This Notebook:** This notebook will explore the Telco customer churn dataset, perform various feature engineering techniques, then train, test, and evaluate several different machine learning models to see which performs the best.

**Contents:**
1. Read and Review Data
2. Feature engineering
3. Split Dataset 
4. Train Models
5. Test and Evaluate Models

In [0]:
# %pip install openpyxl
# %pip install xgboost
# %pip install lightgbm
# %pip install imbalanced-learn

In [0]:
# Essential Libraries
import numpy as np
import pandas as pd

# Libraries for Machine Learning Process
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

# Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# For evaluating a model's predictive performance
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

# 1. Read and review dataset

In [0]:
df = pd.read_excel('data/Telco_customer_churn.xlsx')
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


# 2. Feature engineering

In [0]:
# Make column names lowercase
df.columns = df.columns.str.lower()

# Replace spaces with underscores in column names
df.columns = df.columns.str.replace(" ", "_")

df.head()

,customerid,count,country,state,city,zip_code,lat_long,latitude,longitude,gender,senior_citizen,partner,dependents,tenure_months,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn_label,churn_value,churn_score,cltv,churn_reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [0]:
# Remove seemingly irrelavent columns
dropped_cols = ['count','lat_long', 'latitude', 'longitude', 'payment_method', 'churn_reason', 'churn_label', 'cltv', 'churn_score']
df.drop(columns=dropped_cols, axis=1, inplace=True)

In [0]:
# Get number of rows and columns
df.shape

(7043, 24)

In [0]:
df.head()

,customerid,country,state,city,zip_code,gender,senior_citizen,partner,dependents,tenure_months,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,monthly_charges,total_charges,churn_value
0,3668-QPYBK,United States,California,Los Angeles,90003,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,53.85,108.15,1
1,9237-HQITU,United States,California,Los Angeles,90005,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,70.70,151.65,1
2,9305-CDSKC,United States,California,Los Angeles,90006,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,99.65,820.5,1
3,7892-POOKP,United States,California,Los Angeles,90010,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,104.80,3046.05,1
4,0280-XJGEX,United States,California,Los Angeles,90015,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,103.70,5036.3,1


In [0]:
# Convert total_charges to float data type
df['total_charges'] = pd.to_numeric(df['total_charges'], errors='coerce').fillna(0.0)

In [0]:
# Check for null values
df.isnull().sum()

customerid           0
country              0
state                0
city                 0
zip_code             0
gender               0
senior_citizen       0
partner              0
dependents           0
tenure_months        0
phone_service        0
multiple_lines       0
internet_service     0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_tv         0
streaming_movies     0
contract             0
paperless_billing    0
monthly_charges      0
total_charges        0
churn_value          0
dtype: int64

In [0]:
# Apply logarithmic transformation to numerical columns to make the data more normalized
numerical_columns = ['tenure_months', 'monthly_charges', 'total_charges']

for col in numerical_columns:
    df[col] = np.log1p(df[col].values)

# 3. Use DictVectorizer to one-hot encode categorical features, split dataset, and use SMOTE to help prevent oversampling.

In [0]:
df_X = df.copy()
df_X.drop('churn_value', axis=1, inplace=True)
data_dict = df_X.to_dict(orient='records')
dv = DictVectorizer()
features = dv.fit_transform(data_dict)

In [0]:
# labels — only the target variable column
labels = df['churn_value']

In [0]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(features, labels)

In [0]:
# Create Train and Test Splits
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Report Number and Proportion of Train and Test Features and Labels
print(f'Train Split: {X_train.shape[0]} Records, {len(y_train)} Labels = {round(len(y_train)/len(y_resampled), 4) * 100}%')
print(f'Test Split: {X_test.shape[0]} Records, {len(y_test)} Labels = {round(len(y_test)/len(y_resampled), 4) * 100}%')

Train Split: 8278 Records, 8278 Labels = 80.0%
Test Split: 2070 Records, 2070 Labels = 20.0%


# 4. Train the following models: Logistic Regression, Decision Tree Classifier, Random Forest Classifier, Gradient Boosting Classifier, XGBoost Classifier, LGBM Classifier

In [0]:
params_grid = {
    'LogisticRegression': {
        'solver': 'liblinear',
        'C': 1.0,
        'max_iter': 1000,
        'random_state': 42
    },
    'DecisionTreeClassifier': {},
    'RandomForestClassifier': {
        'n_estimators': 10,
        'max_depth': 20,
        'random_state': 1,
        'n_jobs': 1
    },
    'GradientBoostingClassifier': {},
    'XGBClassifier': {
        'objective': 'binary:logistic',
        'learning_rate': 0.1,
        'max_depth': 3,
        'n_estimators': 100
    },
    'LGBMClassifier': {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'learning_rate': 0.1,
        'max_depth': 5,
        'num_leaves': 31
    }
}

In [0]:
models = [
    LogisticRegression(**params_grid['LogisticRegression']),
    DecisionTreeClassifier(**params_grid['DecisionTreeClassifier']), 
    RandomForestClassifier(**params_grid['RandomForestClassifier']), 
    GradientBoostingClassifier(**params_grid['GradientBoostingClassifier']),
    XGBClassifier(**params_grid['XGBClassifier']),
    LGBMClassifier(**params_grid['LGBMClassifier'])
]

In [0]:
# Train the model using the training features and labels
for model in models:
    model.fit(X_train, y_train)
    # Report trained model
    print(f'Trained and ready: {model}')

Trained and ready: LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')
Trained and ready: DecisionTreeClassifier()
Trained and ready: RandomForestClassifier(max_depth=20, n_estimators=10, n_jobs=1, random_state=1)
Trained and ready: GradientBoostingClassifier()
Trained and ready: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
         

# 5. Evaluate and compare all models using accuracy, precision, recall, roc auc, and f1 score

In [0]:
# Create list of scores and insert into dataframe for easy viewing
scores_list = []

# Test all models on the test split
for model in models:
    # Use the model to generate predictions for the Test split, based on its features only
    y_pred = model.predict(X_test)

    # Compare model's predictive performance to the provided test labels
    accuracy = round(accuracy_score(y_test, y_pred) * 100, 3)
    precision = round(precision_score(y_test, y_pred) * 100, 3)
    recall = round(recall_score(y_test, y_pred) * 100, 3)
    roc_auc = round(roc_auc_score(y_test, y_pred).round(3) * 100, 3)
    f1 = round(f1_score(y_test, y_pred) * 100, 3)

    model_name = str(model).split("(")[0]

    scores_list.append([model_name, accuracy, precision, recall, roc_auc, f1])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [0]:
scores_list

[['LogisticRegression', 85.797, 83.231, 90.332, 85.7, 86.636],
 ['DecisionTreeClassifier', 83.333, 85.714, 80.758, 83.4, 83.163],
 ['RandomForestClassifier', 85.942, 84.727, 88.341, 85.9, 86.497],
 ['GradientBoostingClassifier', 86.812, 87.961, 85.877, 86.8, 86.906],
 ['XGBClassifier', 87.391, 90.02, 84.645, 87.4, 87.25],
 ['LGBMClassifier', 87.391, 89.542, 85.213, 87.4, 87.324]]

In [0]:
df_scores = pd.DataFrame(scores_list, columns = ['model', 'accuracy', 'precision', 'recall', 'roc_auc', 'f1_score'])
df_scores

,model,accuracy,precision,recall,roc_auc,f1_score
0,LogisticRegression,85.797,83.231,90.332,85.7,86.636
1,DecisionTreeClassifier,83.333,85.714,80.758,83.4,83.163
2,RandomForestClassifier,85.942,84.727,88.341,85.9,86.497
3,GradientBoostingClassifier,86.812,87.961,85.877,86.8,86.906
4,XGBClassifier,87.391,90.020,84.645,87.4,87.250
5,LGBMClassifier,87.391,89.542,85.213,87.4,87.324
